# Guild Awards Analysis

<ul>
<li><a href="#garothi">Garothi</a></li>
<li><a href="#enoar">Eonar</a>
    <ul>
        <li><a href="#damage-debuff">Extra Damage Debuff</a></li>
        <li><a href="#air-knock-debuff">Knocked into Air Debuff</a></li>
    </ul>
</li>
</ul>

In [1]:
import pandas as pd
import numpy as np
import requests
import warcraft_logs_fn as wl
import log_analysis as la

In [2]:
api_key = '0a60df4bb184d6c614fb96e0126bd407'

Import log details

In [3]:
df = pd.read_csv('master_list.csv', encoding="iso-8859-1")
df.head()

,log_id,log_start,log_end,pull_id,pull_start,pull_end,boss_id,boss_name,kill,player_name
0,1dApmwBHyQ2vcVTF,1.526609e+12,1.526619e+12,1,1397463,1482607,2092,Argus the Unmaker,False,Petrol
1,1dApmwBHyQ2vcVTF,1.526609e+12,1.526619e+12,1,1397463,1482607,2092,Argus the Unmaker,False,Radiantldeal
2,1dApmwBHyQ2vcVTF,1.526609e+12,1.526619e+12,1,1397463,1482607,2092,Argus the Unmaker,False,Shallowfall
3,1dApmwBHyQ2vcVTF,1.526609e+12,1.526619e+12,1,1397463,1482607,2092,Argus the Unmaker,False,Acturus
4,1dApmwBHyQ2vcVTF,1.526609e+12,1.526619e+12,1,1397463,1482607,2092,Argus the Unmaker,False,Tawñ


In [8]:
df.log_id.nunique()

93

<a id='explore'></a>
## Eonar

Spell IDs: 249016 & 249015

#### Query Tests

Find pull end time for log with Eonar fights

In [9]:
df.query('log_id == "vYDj1qQgcXWdmTnF"').pull_end.max()

10305202

Run query for debuff on targets for log

In [12]:
debuff = requests.get('https://www.warcraftlogs.com:443/v1/report/tables/debuffs/vYDj1qQgcXWdmTnF?end=10305202&by=source&abilityid=249016&api_key=' + api_key)
details = debuff.json()
details

{'auras': [{'name': 'Èllipses',
   'id': 3,
   'guid': 137327868,
   'type': 'Warlock',
   'icon': 'Warlock-Affliction',
   'totalUptime': 392648,
   'totalUses': 2,
   'bands': [{'startTime': 1587803, 'endTime': 1779949},
    {'startTime': 2285588, 'endTime': 2486090}]},
  {'name': 'Ayriea',
   'id': 17,
   'guid': 132573980,
   'type': 'Hunter',
   'icon': 'Hunter-BeastMastery',
   'totalUptime': 606958,
   'totalUses': 3,
   'bands': [{'startTime': 1472757, 'endTime': 1779949},
    {'startTime': 2186324, 'endTime': 2486090}]},
  {'name': 'Petrol',
   'id': 7,
   'guid': 110737162,
   'type': 'Mage',
   'icon': 'Mage-Fire',
   'totalUptime': 885889,
   'totalUses': 2,
   'bands': [{'startTime': 1331271, 'endTime': 1779949},
    {'startTime': 2048879, 'endTime': 2486090}]},
  {'name': 'Undertakerop',
   'id': 25,
   'guid': 157445886,
   'type': 'DeathKnight',
   'icon': 'DeathKnight-Frost',
   'totalUptime': 175560,
   'totalUses': 4,
   'bands': [{'startTime': 1687311, 'endTime': 17

In [13]:
details.keys()

dict_keys(['auras', 'useTargets', 'totalTime', 'startTime', 'endTime'])

Get name and counts

In [14]:
details['auras'][0]['name']

'Èllipses'

In [15]:
details['auras'][0]['totalUses']

2

#### Create function

In [23]:
for log in df['log_id'].unique():
    print(log)
    break

1dApmwBHyQ2vcVTF


In [26]:
df[df['log_id'] == '1dApmwBHyQ2vcVTF'].pull_end.max()

10679403

In [51]:
def buff_count(api_key, log_df, spell_id, spell_name=None, description=None, buff=True):
    df_list = []
    logs = log_df['log_id'].unique()
    for log in logs:
        print('Collecting details for log', log)
        
        # Create link
        end = df[df['log_id'] == log].pull_end.max()
        if buff is True:
            link = 'https://www.warcraftlogs.com:443/v1/report/tables/buffs/' + log + '?end=' + str(end) + \
            '&by=source&abilityid=' + str(spell_id) + '&api_key='
        else:
            link = 'https://www.warcraftlogs.com:443/v1/report/tables/debuffs/' + log + '?end=' + str(end) + \
            '&by=source&abilityid=' + str(spell_id) + '&api_key='
        
        # Complete query
        event = requests.get(link + api_key)
        details = event.json()
        
        # Get player info
        for player in details['auras']:
            name = player['name']
            print('Player added:', name)
            duration = player['totalUptime']
            df_list.append({
                'log_id': log, 
                'spell_id': spell_id,
                'spell_name': spell_name,
                'description': description,
                'player': name,
                'duration': duration
            })
    return pd.DataFrame(df_list, columns=['log_id', 'spell_id', 'spell_name', 'description', 'player', 'duration'])

In [11]:
test = df.query('log_id == "vYDj1qQgcXWdmTnF"')

In [18]:
extra_damage = debuff_count(api_key, test, 249016, 'Feedback - Targeted', 'extra damage taken', buff=False)

Player added: Ayriea
Player added: Petrol
Player added: Èllipses
Player added: Undertakerop


In [19]:
extra_damage.head()

,log_id,debuff_id,debuff_name,debuff_description,player,debuff_duration
0,vYDj1qQgcXWdmTnF,249016,Feedback - Targeted,extra damage taken,Ayriea,606958
1,vYDj1qQgcXWdmTnF,249016,Feedback - Targeted,extra damage taken,Petrol,885889
2,vYDj1qQgcXWdmTnF,249016,Feedback - Targeted,extra damage taken,Èllipses,392648
3,vYDj1qQgcXWdmTnF,249016,Feedback - Targeted,extra damage taken,Undertakerop,175560


In [21]:
extra_damage = debuff_count(api_key, df, 249016, 'Feedback - Targeted', 'extra damage taken')

Player added: Tinytiki
Player added: Shadowbaine
Player added: Petrol
Player added: Ayriea
Player added: Brian
Player added: Stradivarus
Player added: Bryiah
Player added: Petrol
Player added: Verle
Player added: Illestra
Player added: Bryiah
Player added: Petrol
Player added: Ayriea
Player added: Pulsè
Player added: Thebadlock
Player added: Bryiah
Player added: Ayriea
Player added: Petrol
Player added: Radiantldeal
Player added: Brian
Player added: Èllipses
Player added: Ayriea
Player added: Shadowbaine
Player added: Kamer
Player added: Psychodruid
Player added: Thebadlock
Player added: Au
Player added: Future
Player added: Ayriea
Player added: Pulsè
Player added: Shadowbaine
Player added: Brian
Player added: Acturus
Player added: Kamer
Player added: Shadowbaine
Player added: Petrol
Player added: Tinytiki
Player added: Shallowfall
Player added: Ayriea
Player added: Petrol
Player added: Petrol
Player added: Future
Player added: Au
Player added: Yoked
Player added: Shadowbaine
Player ad

In [22]:
extra_damage.head()

,log_id,debuff_id,debuff_name,debuff_description,player,debuff_duration
0,2c6whdrGDkKHP7Nv,249016,Feedback - Targeted,extra damage taken,Tinytiki,329352
1,2c6whdrGDkKHP7Nv,249016,Feedback - Targeted,extra damage taken,Shadowbaine,439981
2,2c6whdrGDkKHP7Nv,249016,Feedback - Targeted,extra damage taken,Petrol,343314
3,2c6whdrGDkKHP7Nv,249016,Feedback - Targeted,extra damage taken,Ayriea,536246
4,2c6whdrGDkKHP7Nv,249016,Feedback - Targeted,extra damage taken,Brian,89248


In [23]:
extra_damage.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 6 columns):
log_id                64 non-null object
debuff_id             64 non-null int64
debuff_name           64 non-null object
debuff_description    64 non-null object
player                64 non-null object
debuff_duration       64 non-null int64
dtypes: int64(2), object(4)
memory usage: 3.1+ KB


In [34]:
total_extra_damage = extra_damage.groupby('player').debuff_duration.sum().sort_values(ascending=False)
total_extra_damage = pd.DataFrame(total_extra_damage)
total_extra_damage.reset_index(inplace=True)
total_extra_damage

,player,debuff_duration
0,Petrol,10688003
1,Ayriea,8304100
2,Tinytiki,2433008
3,Èllipses,2353682
4,Shadowbaine,2088836
5,Psychodruid,1651214
6,Bryiah,1297521
7,Shallowfall,949497
8,Kamer,744238
9,Future,642201


In [27]:
knock_back = debuff_count(api_key, df, 249015, 'Feedback - Burning Embers', 'knock into air')

Player added: Kyarrix
Player added: Shadowbaine
Player added: Metonymy
Player added: Znoch
Player added: Petrol
Player added: Shadowbaine
Player added: Psychodruid
Player added: Metonymy
Player added: Brian
Player added: Brian
Player added: Shadowbaine
Player added: Kyarrix
Player added: Metonymy
Player added: Shadowbaine
Player added: Brian
Player added: Psychodruid
Player added: Metonymy
Player added: Znoch
Player added: Kyarrix
Player added: Petrol
Player added: Metonymy
Player added: Znoch
Player added: Petrol
Player added: Brianmurican
Player added: Metonymy
Player added: Metonymy
Player added: Kyarrix
Player added: Petrol
Player added: Psychodruid
Player added: Psychodruid
Player added: Metonymy
Player added: Petrol
Player added: Kyarrix
Player added: Metonymy
Player added: Psychodruid
Player added: Brianmurican
Player added: Znoch
Player added: Znoch
Player added: Kyarrix
Player added: Psychodruid
Player added: Brian
Player added: Pulsè
Player added: Èllipses
Player added: Meton

In [28]:
knock_back.head()

,log_id,debuff_id,debuff_name,debuff_description,player,debuff_duration
0,2c6whdrGDkKHP7Nv,249015,Feedback - Burning Embers,knock into air,Kyarrix,326939
1,2c6whdrGDkKHP7Nv,249015,Feedback - Burning Embers,knock into air,Shadowbaine,339102
2,2c6whdrGDkKHP7Nv,249015,Feedback - Burning Embers,knock into air,Metonymy,535104
3,2c6whdrGDkKHP7Nv,249015,Feedback - Burning Embers,knock into air,Znoch,92884
4,2c6whdrGDkKHP7Nv,249015,Feedback - Burning Embers,knock into air,Petrol,445989


In [33]:
total_knock_back = knock_back.groupby('player').debuff_duration.sum().sort_values(ascending=False)
total_knock_back = pd.DataFrame(total_knock_back)
total_knock_back.reset_index(inplace=True)
total_knock_back

,player,debuff_duration
0,Metonymy,9949922
1,Znoch,5962057
2,Shadowbaine,4539189
3,Petrol,4415304
4,Brian,3089607
5,Brianmurican,2296808
6,Psychodruid,1445497
7,Kyarrix,1258462
8,Èllipses,836739
9,Pulsè,588255


In [35]:
combined_df = pd.merge(left=total_extra_damage, right=total_knock_back, how='outer', on='player')
combined_df

,player,debuff_duration_x,debuff_duration_y
0,Petrol,10688003.0,4415304.0
1,Ayriea,8304100.0,NaN
2,Tinytiki,2433008.0,NaN
3,Èllipses,2353682.0,836739.0
4,Shadowbaine,2088836.0,4539189.0
5,Psychodruid,1651214.0,1445497.0
6,Bryiah,1297521.0,NaN
7,Shallowfall,949497.0,NaN
8,Kamer,744238.0,NaN
9,Future,642201.0,NaN


In [36]:
combined_df.rename(columns={'debuff_duration_x': 'extra_damage_duration', 
                            'debuff_duration_y': 'knock_back_duration'}, 
                   inplace=True)

In [37]:
combined_df.head()

,player,extra_damage_duration,knock_back_duration
0,Petrol,10688003.0,4415304.0
1,Ayriea,8304100.0,NaN
2,Tinytiki,2433008.0,NaN
3,Èllipses,2353682.0,836739.0
4,Shadowbaine,2088836.0,4539189.0


In [41]:
combined_df.fillna(0, inplace=True)

In [44]:
combined_df['total_duration'] = combined_df.extra_damage_duration + combined_df.knock_back_duration
combined_df.sort_values('total_duration', ascending=False, inplace=True)
combined_df

,player,extra_damage_duration,knock_back_duration,total_duration
0,Petrol,10688003.0,4415304.0,15103307.0
22,Metonymy,0.0,9949922.0,9949922.0
1,Ayriea,8304100.0,0.0,8304100.0
4,Shadowbaine,2088836.0,4539189.0,6628025.0
23,Znoch,0.0,5962057.0,5962057.0
17,Brian,238977.0,3089607.0,3328584.0
3,Èllipses,2353682.0,836739.0,3190421.0
5,Psychodruid,1651214.0,1445497.0,3096711.0
2,Tinytiki,2433008.0,0.0,2433008.0
24,Brianmurican,0.0,2296808.0,2296808.0


In [46]:
duration = combined_df.query('player == "Petrol"').total_duration[0]
duration

15103307.0

In [50]:
duration/(60*60)/1000

4.195363055555555